In [1]:
#라이브러리
import numpy as np
import pandas as pd


In [5]:
#전처리한 데이터를 불러온다.
df = pd.read_csv('./Data/diabetes_feature.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,0,33.6,0.627,50,1,False,False,True,False,206.0,5.332719,False
1,1,85,66,29,0,26.6,0.351,31,0,False,False,True,False,130.0,4.875197,False
2,8,183,64,0,0,23.3,0.672,32,1,True,False,True,False,206.0,5.332719,False
3,1,89,66,23,94,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [6]:
#회귀로 구한 인슐린 값을 가져온다.
df_insulin=  pd.read_csv('./Data/diabetes_fill_insulin.csv')
df_insulin.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,168.0,33.6,0.627,50,1
1,1,85,66,29,37.0,26.6,0.351,31,0
2,8,183,64,0,130.0,23.3,0.672,32,1
3,1,89,66,23,94.0,28.1,0.167,21,0
4,0,137,40,35,168.0,43.1,2.288,33,1


In [7]:
#전처리한 데이터에 구한 인슐린 값을 넣어준다.
df['Insulin'] = df_insulin['Insulin']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,168.0,33.6,0.627,50,1,False,False,True,False,206.0,5.332719,False
1,1,85,66,29,37.0,26.6,0.351,31,0,False,False,True,False,130.0,4.875197,False
2,8,183,64,0,130.0,23.3,0.672,32,1,True,False,True,False,206.0,5.332719,False
3,1,89,66,23,94.0,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.0,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


#### 학습과 예측에 사용할 데이터 만들기

In [8]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [14]:
X = df[[ 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Pregnancies_high',
       'Age']]
X.shape

(768, 8)

In [15]:
y= df['Outcome']
y.shape

(768,)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test = train_test_split(
    X,y,test_size =0.2,random_state =42)

In [19]:
X_train.shape, y_train.shape

((614, 8), (614,))

#### 최적의 모델 구하기

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [21]:
estimators = [DecisionTreeClassifier(random_state =42),
            RandomForestClassifier(random_state=42),
            GradientBoostingClassifier(random_state =42)]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [27]:
#RandomSearchCV 이용하기

from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2,20,10)
max_features = np.random.uniform(0.3,1,10)

param_distributions = {'max_depth' : max_depth,
                      'max_features' : max_features}

results =[]
for estimator in estimators:
    result =[]
    if estimator.__class__.__name__ !='DecisionTreeClassifier':
        param_distributions['n_estimators']=np.random.randint(100,200,10)
    
    clf= RandomizedSearchCV(estimator,
                           param_distributions,
                           n_iter =10,
                           scoring ='accuracy',
                           n_jobs =-1,
                           cv =5,
                           verbose = 0)
    
    clf.fit(X_train, y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test,y_test))
    result.append(clf.cv_results_)
    results.append(result)

In [29]:
df= pd.DataFrame(results,
                columns =["estimator","best_params","train_score","test_score","cv_result"])
df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.9777209089256516, 'max_dept...",0.755738,0.772727,"{'mean_fit_time': [0.009574222564697265, 0.010..."
1,RandomForestClassifier,"{'n_estimators': 129, 'max_features': 0.474786...",0.768772,0.746753,"{'mean_fit_time': [0.5524362087249756, 0.93505..."
2,GradientBoostingClassifier,"{'n_estimators': 101, 'max_features': 0.977720...",0.770385,0.727273,"{'mean_fit_time': [0.4892903804779053, 0.35359..."
